# Assignment 3: Nonlinear Programming

## Instructions:
1. Answer the questions in this Jupyter Notebook file and upload it in the Assignment 3 entry under the Assignments tab in MyCourses. Answer each question in its place. Late submissions are NOT accepted. Also, files not in the correct format (other than a Jupyter Notebook) are NOT accepted.
2. You can submit your answer file multiple times, but only the last submission is kept and graded.
3. For each question, print all the results that the question asks. Also, prevent printing unnecessary information.
4. Your submissions will be graded based on the correctness of the results.
5. For formulation problems, the only acceptable answers are the correct formulas. If you are using an additional variable, introduce it clearly.
6. **Unless otherwise stated, whenever we mention the "risk," we mean the standard deviation and not the variance.**

In [1]:
import pandas as pd
import numpy as np

import gurobipy as gb
from gurobipy import *

# Problem 1: The Markowitz Problem (50 points)

## The Story:

A rational investor wishes for a low variance on return and a high expected rate of return. However, the two goals seldom align. For example, you can always keep variance down by investing in bonds over stocks, but you do so at the expense of a decent rate of return. So, an investor's optimal portfolio could be described by ($r, \sigma$), where $r$ is a desired (and feasible) average rate of return, and $\sigma$ is the minimal standard deviation possible for this given $r$. (Put differently, an investor might wish to find the highest rate of return possible for a given acceptable level of risk.) This problem is due originally to Harry Markowitz in the 1950's.

## The Data:

The data in the Excel file "Markowitz" provides the performances of five stock exchange indices from 31 Dec 2010 to 1 June 2016. Throughout this problem, we will use that dataset to find the optimal portfolio. But before that, we go through some warm-up questions.

### Caution:

In this problem, do not convert the values to percentages. Using percentages can lead to wrong results if you're not cautious.

## Question 1: (5 points)
Use the following code to calculate the monthly rate of return (RoR) of each index. 

What does *pct_change(-1)* in the following code do?

In [2]:
StocksData = pd.read_excel('Markowitz.xlsx', header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of returns (RoRs).
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row. (The earliest date doesn't have any data before to be compared with.)
rate_of_return_df.dropna(inplace = True)

display(rate_of_return_df)

,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
2016-06-01,-0.042370,-0.063923,-0.008266,-0.016583,-0.004754
2016-05-03,-0.001778,0.022290,0.000763,-0.013982,0.021170
2016-04-01,0.010850,0.007371,0.005007,0.017413,0.015098
2016-03-01,0.012760,0.049509,0.070753,0.080918,0.077503
2016-02-01,0.002186,-0.030895,0.003049,-0.039002,-0.001429
...,...,...,...,...,...
2011-05-02,-0.013163,-0.029379,-0.018793,-0.026757,-0.019635
2011-04-01,0.027264,0.067196,0.039839,0.015772,0.025772
2011-03-01,-0.014214,-0.031766,0.007638,-0.047549,0.024409
2011-02-01,0.022361,0.027530,0.028121,0.030477,0.054016


*Your answer here*:

- The pct_change function in pandas calculates the percentage difference between the current and a preceding element. When employed with the argument (-1), it computes the percentage change relative to the previous row, effectively shifting the comparison one period backward.
- In pandas, the default behavior of the pct_change method involves multiplying the result by 100 to represent the change as a percentage. However, when using pct_change() without any additional parameters, it provides the fractional change, which is the appropriate format for rate of returns in financial analyses. As a result, it can be utilized directly without the need to convert the values into percentages.

## Question 2: (10 points)

Estimate and return **average, variance, and standard deviation of the RoR for each index in that period**. Then, estimate and return **the covariance and correlation matrix between each pair of indices in that period**. (Hint: You can use the *mean()*, *var()*, *cov()*, and *corr()* functions from the Pandas package.)

Hint: If done correctly, you must have:
1. The average monthly return of Russell 2000 is 0.0069.
2. The covariance between DAX and DJIA is 0.001224.

In [3]:
StocksData = pd.read_excel('Markowitz.xlsx', header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of returns (RoRs).
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row. (The earliest date doesn't have any data before to be compared with.)
rate_of_return_df.dropna(inplace = True)

#############################################
# Your code here:
## Metrics calculated below are for each index
# Average rate of return
average_returns = rate_of_return_df.mean()

# Variance of the rate of return
variance_returns = rate_of_return_df.var()

# Standard deviation of the rate of return
stddev_returns = rate_of_return_df.std()

# Covariance matrix (pair of indices)
covariance_matrix = rate_of_return_df.cov()

# Correlation matrix (pair of indices)
correlation_matrix = rate_of_return_df.corr()

# Results
print(average_returns)
print(variance_returns)
print(stddev_returns)
print(covariance_matrix)
print(correlation_matrix)

Average Returns:
FTSE 100              0.000695
DAX                   0.006402
DJIA                  0.006931
DJ Asian Titans 50   -0.001075
Russell 2000          0.006875
dtype: float64

Variance of Returns:
FTSE 100              0.001059
DAX                   0.002794
DJIA                  0.001082
DJ Asian Titans 50    0.002045
Russell 2000          0.002147
dtype: float64

Standard Deviation of Returns:
FTSE 100              0.032549
DAX                   0.052861
DJIA                  0.032890
DJ Asian Titans 50    0.045223
Russell 2000          0.046334
dtype: float64

Covariance Matrix:
                    FTSE 100       DAX      DJIA  DJ Asian Titans 50  \
FTSE 100            0.001059  0.001299  0.000827            0.001012   
DAX                 0.001299  0.002794  0.001224            0.001597   
DJIA                0.000827  0.001224  0.001082            0.001073   
DJ Asian Titans 50  0.001012  0.001597  0.001073            0.002045   
Russell 2000        0.000980  0.001700 

## Question 3: (10 points)

Formulate an NLP to devise the optimal portfolio that minimizes the portfolio risk (the standard deviation) subject to non-negative expected return. In your formulation, remember to include the constraint that the sum of the weights of the stocks in the portfolio should be equal to 1, and the weights should be non-negative. 

What are **the optimal portfolio, the optimal risk, and the expected return of the optimal portfolio?** 

Hint 1: This problem is an implementation of the "Risk Minimization Formulation" you have seen in the class. \
Hint 2: Minimizing the variance is equivalent to minimizing the standard deviation, as the variance is the square of the standard deviation, an increasing function. You may use this fact because minimizing variance is more straightforward to code than minimizing standard deviation. Nevertheless, report the standard deviation and not the variance in your answer.

In [6]:
StocksData = pd.read_excel('Markowitz.xlsx', header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of returns (RoRs).
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row. (The earliest date doesn't have any data before to be compared with.)
rate_of_return_df.dropna(inplace = True)

# Calculate the average rate of return for each stock.
RoR_Avg = rate_of_return_df.mean()

# Calculate the covariance between each pair of stocks.
RoR_Cov = rate_of_return_df.cov()

###################################################################
# Create a new model.
model = gb.Model('Question3')
model.Params.LogToConsole = 0
m = len(Stock_Name)

# Your code here:

# Variables
# Weights variables of the stocks
weights = model.addVars(m, lb=0, ub=1, name='weights')

# Quadratic expression of portfolio variance
portfolio_variance = gb.QuadExpr()
for i in range(m):
    for j in range(m):  # Loop through the covariance matrix
        # Since the covariance matrix is symmetric, we can simply multiply i,j and j,i by the weight variables
        portfolio_variance += RoR_Cov.iloc[i, j] * weights[i] * weights[j]

# Objective Funtion (Min. Portfolio Variance)
model.setObjective(portfolio_variance, gb.GRB.MINIMIZE)

# First Constraint: Sum of weights = 1
model.addConstr(weights.sum() == 1, 'sum of weights')

# Second Constraint: Non-negativity of expected return
model.addConstr(sum(RoR_Avg[i] * weights[i] for i in range(m)) >= 0, 'non-negative return')


model.optimize()
optimal_weights = model.getAttr('x', weights)

# Optimal portfolio's expected return
optimal_return = sum(RoR_Avg[i] * optimal_weights[i] for i in range(m))

# Optimal portfolio's variance & std deviation
optimal_variance = portfolio_variance.getValue()
optimal_risk = np.sqrt(optimal_variance)

print("Optimal Weights:", optimal_weights)
print("Expected Return of the Optimal Portfolio:", optimal_return)
print("Risk (Standard Deviation) of the Optimal Portfolio:", optimal_risk)

Optimal Weights: {0: 0.5229355925460748, 1: 9.675646189984122e-07, 2: 0.4770333238336418, 3: 2.1768729671716412e-05, 4: 8.347325980267647e-06}
Expected Return of the Optimal Portfolio: 0.0036696582365349207
Risk (Standard Deviation) of the Optimal Portfolio: 0.03080125464473466


## Question 4: (10 points)

Model the problem of finding the optimal portfolio that maximizes the expected return subject to the constraint that the portfolio risk is less than or equal to 0.04. In your formulation, do not forget to include the constraint that the sum of the weights of the stocks in the portfolio should be equal to 1, and the weights should be non-negative. 

What is the **optimal portfolio, the optimal expected return, and the risk of the optimal portfolio** (Hint: This problem is an implementation of "Return Maximization Formulation" that you have seen in the class)

In [7]:
StocksData = pd.read_excel('Markowitz.xlsx', header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of returns (RoRs).
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row. (The earliest date doesn't have any data before to be compared with.)
rate_of_return_df.dropna(inplace = True)

# Calculate the average rate of return for each stock.
RoR_Avg = rate_of_return_df.mean()

# Calculate the covariance between each pair of stocks.
RoR_Cov = rate_of_return_df.cov()

###################################################################
# Create a new model.
model = gb.Model('Question4')
model.Params.LogToConsole = 0
m = len(Stock_Name)

# Your code here

# Variables
# Weights variables of the stocks
weights = model.addVars(m, lb=0, ub=1, name='weights')

# Objective Funtion (Max. Portfolio Expected Return)
model.setObjective(sum(RoR_Avg[i] * weights[i] for i in range(m)), gb.GRB.MAXIMIZE)


# First Constraint: Sum of weights = 1
model.addConstr(weights.sum() == 1, 'sum of weights')

# Second Constraint: The portfolio risk (std deviation) should be less than or equal to the risk limit
risk_limit = 0.04
portfolio_variance = gb.QuadExpr()
for i in range(m):
    for j in range(m):  # Loop through the covariance matrix
        portfolio_variance += RoR_Cov.iloc[i, j] * weights[i] * weights[j]


model.addConstr(portfolio_variance <= risk_limit**2, 'risk constraint')
model.optimize()
optimal_weights = model.getAttr('x', weights)

# Optimal portfolio's expected return
optimal_return = sum(RoR_Avg[i] * optimal_weights[i] for i in range(m))

# Optimal portfolio's variance and std deviation (risk)
optimal_variance = portfolio_variance.getValue()
optimal_risk = np.sqrt(optimal_variance)

print("Optimal Weights:", optimal_weights)
print("Optimal Expected Return of the Portfolio:", optimal_return)
print("Risk (Standard Deviation) of the Optimal Portfolio:", optimal_risk)

Optimal Weights: {0: 2.6811596656073213e-09, 1: 4.163312687071582e-08, 2: 0.9999924666572866, 3: 2.1540928782364468e-09, 4: 7.486877004070185e-06}
Optimal Expected Return of the Portfolio: 0.0069308401478900775
Risk (Standard Deviation) of the Optimal Portfolio: 0.03289003881846509


## Question 5: (10 points)

We want to implement one of the practical extensions of the Markowitz problem discussed in class. In particular, we want to implement **the transaction costs**. As a portfolio manager, you rarely build a portfolio from scratch. Instead, you start with an existing portfolio and make some changes. For example, you can sell some of the stocks in your portfolio and buy others. In this case, you have to pay some transaction costs. Here, we assume you are given a portfolio with 20% of your money invested in each of the five indices. You may want to change this portfolio to a new portfolio, but you have to pay the transaction costs.

Here, we want to maximize the expected return minus the transaction costs such that the risk of the new portfolio is less than or equal to 0.04. Therefore, the objective function is:
$$\sum_{i = 1}^5 R_i \times x_i - 0.1 \times \sum_{i = 1}^5 (x_i - 0.2)^2,$$
Where $x_i$ is the weight of the *i*th index in the new portfolio, and $R_i$ is the average return of the *i*th index.

In [8]:
StocksData = pd.read_excel('Markowitz.xlsx', header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of returns (RoRs).
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row. (The earliest date doesn't have any data before to be compared with.)
rate_of_return_df.dropna(inplace = True)

# Calculate the average rate of return for each stock.
RoR_Avg = rate_of_return_df.mean()

# Calculate the covariance between each pair of stocks.
RoR_Cov = rate_of_return_df.cov()

###################################################################
# Create a new model.
model = gb.Model('Question5')
model.Params.LogToConsole = 0
m = len(Stock_Name)

# Your code here

# Variables
weights = model.addVars(m, lb=0, ub=1, name='weights')

# Portfolio weights (20% for each index)
current_weights = np.array([0.2] * m)

# Objective Function (Max. the expected return - transaction costs)
objective = gb.quicksum(RoR_Avg[i] * weights[i] for i in range(m)) \
            - 0.1 * gb.quicksum((weights[i] - current_weights[i]) ** 2 for i in range(m))
model.setObjective(objective, gb.GRB.MAXIMIZE)

# First Constraint: Sum of weights = 1
model.addConstr(weights.sum() == 1, 'sum of weights')

# Second Constraint: The portfolio risk (std deviation) should be less than or equal to the risk limit
risk_limit = 0.04
portfolio_variance = gb.QuadExpr()
for i in range(m):
    for j in range(m):  # Loop through the covariance matrix
        portfolio_variance += RoR_Cov.iloc[i, j] * weights[i] * weights[j]

model.addConstr(portfolio_variance <= risk_limit**2, 'risk constraint')

model.optimize()
optimal_weights = model.getAttr('x', weights)

# Optimal portfolio's expected return
optimal_return = sum(RoR_Avg[i] * optimal_weights[i] for i in range(m))

# Transaction costs
transaction_costs = 0.1 * sum((optimal_weights[i] - current_weights[i]) ** 2 for i in range(m))

# Net expected return after considering transaction costs
net_optimal_return = optimal_return - transaction_costs

# Optimal portfolio's variance and std deviation (risk)
optimal_variance = portfolio_variance.getValue()
optimal_risk = np.sqrt(optimal_variance)

print("Optimal Weights:", optimal_weights)
print("Optimal Expected Return of the Portfolio (before costs):", optimal_return)
print("Transaction Costs:", transaction_costs)
print("Net Optimal Expected Return (after costs):", net_optimal_return)
print("Risk (Standard Deviation) of the Optimal Portfolio:", optimal_risk)

Optimal Weights: {0: 0.18364247229126124, 1: 0.21249419265723213, 2: 0.21441323219034003, 3: 0.17482719726751167, 4: 0.21462290561621433}
Optimal Expected Return of the Portfolio (before costs): 0.004261586781263537
Transaction Costs: 0.00014789141911403285
Net Optimal Expected Return (after costs): 0.004113695362149504
Risk (Standard Deviation) of the Optimal Portfolio: 0.03703982192508558


## Question 6: (5 points)

In class, you saw that it is necessary to set *model.params.NonConvex = 2* to solve some nonlinear problems. However, this was not the case for Problem 1 in this assignment. Why?

*Your answer here*:
#### When we use model.params.NonConvex parameter?
- In Gurobi, when confronted with models containing non-convex quadratic constraints or objectives, denoted as the scenario discussed in our class, the parameter model.params.NonConvex is configured to 2. This specific setting instructs Gurobi to employ techniques designed for tackling non-convex quadratic problems. Such problems are typically more challenging to solve and demand increased computational resources compared to their convex counterparts.
#### Why this parameter is not needed for problem 1 of this assignment?
- In problem 1, we address a convex quadratic optimization problem aiming to minimize portfolio variance. The convexity stems from the objective function, minimizing variance, which is inherently convex in portfolio weights due to the positive semi-definite nature of the covariance matrix. Linear constraints (sum of weights equals 1, non-negative weights) further maintain the convexity. When faced with non-convex constraints, setting model.params.NonConvex to 2 is necessary for problem resolution.

# Problem 2: (50 points)

## The Story and Data

John Brooks, managing director of the CYCOM Corporation, is trying to determine how to allocate CYCOM's research and development budget for the coming year. Six different projects are under consideration. John believes that the success of every project depends in part on the number of engineers assigned to the project. Let $x_1, ..., x_6$ denote the number of engineers assigned to projects 1,...,6. According to internal corporate estimates, the probability that each project will be successful is given according to the function in the third column of Table 1:


| Project | Start-up Cost (in \$1000) | Probability of Success | Profit if Successful (in \$1000) |
| --- | --- | --- | --- |
| 1 | 325 | $\frac{x_1}{x_1 + 1.1}$ | 1,750 |
| 2 | 200 | $\frac{x_2}{x_2 + 0.5}$ | 700 |
| 3 | 490 | $\frac{x_3}{x_3 + 2.5}$ | 1,300 |
| 4 | 90 | $\frac{x_4}{x_4 + 1.6}$ | 800 |
| 5 | 210 | $\frac{x_5}{x_5 + 2.2}$ | 1,450 |
| 6 | 150 | $\frac{x_6}{x_6 + 2.4}$ | 1,300 |


For example, if 10 engineers are assigned to project 1, the probability of success in that project is $\frac{10}{10 + 1.1} = 0.901$. Cycom can assign up to 25 engineers to these six projects. The cost of an engineer is $\$150,000$ (per year). An engineer can be assigned to more than one project. For example, $x_2$ = 3.4 means that 3.4 engineers are assigned to Project 2, which means that 3 engineers are assigned to this project full-time, and one engineer is assigned to this project only 40% of the time. Nonetheless, the total number of hired engineers should be an integer less than or equal to 25.

If a project is unsuccessful, it incurs its full start-up cost, shown in the second column of Table 1. If the project is successful, it realizes the profit value shown in the fourth column of Table 1. This value includes the start-up cost of the project (but does not include the cost of engineers). Thus, for example, if Project 1 is unsuccessful, it incurs a cost of $\$325,000$. If Project 1 is successful, it will realize a profit of $\$1,750,000$.

In this problem, we assume that the manager does not have the option of abandoning a project to prevent its start-up cost. The only thing the manager can choose is the number of engineers assigned to each project.

## Question 7: The Expected Profit Approach

As a business, CYCOM wants to maximize its total expected profit. The total expected profit is the sum of the expected profit of each project minus the total cost of the engineers. The expected profit of each project is the probability of its success times the profit if successful minus the probability of its failure multiplied by the start-up cost. For instance, suppose the probability of success for Project 1 is $P_1$. Then, the expected profit for Project 1 is $P_1 \times 1,750,000 - (1 - P_1) \times 325,000$. 

### Question 7.1: The Formulation (10 points)

Complete the formulation of the problem of maximizing the total expected profit of CYCOM. You may use the following values as given: \
$b_i$: profit if project $i$ is successful. \
$c_i$: start-up cost of project $i$. \
$a_i$: the constant term in the denominator of the probability of success of project $i$.

### Decision Variables:
$x_i$: The number of engineers assigned to project $i$ - non-negative continuous variables. \
$y$: Total number of engineers hired - non-negative integer variable.

### Objective Function:
- Expected Profit to be Maximized:

$\sum_{i = 1}^6 \left(\frac{x_i}{x_i + a_i} \times b_i - \left(1 - \frac{x_i}{x_i + a_i}\right) \times c_i\right) - 150,000 \times y$

Considering the variable $$P_i$$ as follows: $$P_i=\frac{x_i}{(x_i + a_i)}$$
### Constraints:
 $$\sum_{i = 1}^6 x_i \leq y_i$$
 $$ y \leq 25 $$
 $$  x_i \geq 0 (for \: i = 1, \dots, 6)$$

### Question 7.2: The Code (10 points)

Implement the formulation you introduced in Question 7.1 in Python. Then, **find and report the optimal total expected profit of CYCOM, total number of engineers, and the number of engineers assigned to each project.**

**Technical Note 1:**  Unlike solving linear programs with Gurobi, where **scaling** did not matter that much, in nonlinear programs, the scale matters. For instance, if this were a linear program, using $\$150,000$ or $\$150$ as the cost of an engineer would not make any difference. However, in NLPs, using $\$150,000$ instead of $\$150$ can make the problem much more complex for Gurobi. Therefore, I recommend using the costs in thousands of dollars instead of dollars.

**Technical Note 2:** Gurobi does not allow you to introduce constraints/objective functions in fractional format. Think about how you can convert the fractional constraints/objective function to linear constraints/objective function. Maybe introducing a new variable $p_i$ and setting $p_i . (x_i + a_i) = x_i$ can help you.

In [2]:
SetupCost = [325, 200, 490, 125, 710, 240]
Profit = [1750, 700, 1300, 800, 1450, 1300]
Denom = [1.1, 0.5, 2.5, 1.6, 2.2, 2.4]
n = len(Profit)

# Create a new model.
model = gb.Model('Question7')
model.Params.LogToConsole = 0
model.params.NonConvex = 2

# Your code here:

# Variables
x = model.addVars(n, lb=0, name="Engineers assigned to each project") 
y = model.addVar(vtype=gb.GRB.INTEGER, lb=0, name="Total engineers")  
p = model.addVars(n, name="p")  # New variable for fractional part

# Objective Function (Max. Expected Profit)
objective = gb.quicksum(p[i] * Profit[i] - (1 - p[i]) * SetupCost[i] for i in range(n)) - 150 * y
model.setObjective(objective, gb.GRB.MAXIMIZE)

# Constraints
# First Constraint: Sum of engineers assigned should not exceed the total number of engineers
model.addConstr(sum(x[i] for i in range(n)) <= y)

# Second Constraint: The total number of engineers should be less than or equal to 25
model.addConstr(y <= 25)

# Constraints for the new variables p_i
for i in range(n):
    model.addConstr(p[i] * (x[i] + Denom[i]) == x[i])

model.optimize()

if model.status == gb.GRB.OPTIMAL:
    optimal_profit = model.objVal
    total_engineers = y.X
    engineers_per_project = [x[i].X for i in range(n)]
    print("Optimal Total Expected Profit (in $1000):", optimal_profit)
    print("Total Number of Engineers:", total_engineers)
    print("Number of Engineers Assigned to Each Project:", engineers_per_project)

Restricted license - for non-production use only - expires 2024-10-28
Optimal Total Expected Profit (in $1000): 1395.1642368463645
Total Number of Engineers: 15.0
Number of Engineers Assigned to Each Project: [2.8718196285565383, 1.2649463466285014, 3.067078530230634, 1.6044347019653824, 3.5335594104738712, 2.6581613821450745]


## Question 8: Minimizing the Uncertainty Approach

Similar to the stock market problem, we can also use the uncertainty approach to solve this problem. In this approach, we want to minimize the uncertainty (the standard deviation) in the total profit of CYCOM. For this, suppose the success or failure of each project is independent of the success or failure of other projects. Then, the variance of the total expected profit of CYCOM is the sum of the variances of the expected profits of each project. Although we want to decrease the uncertainty in the total profit of CYCOM, we want the expected profit of CYCOM to be at least $1.1 million.

**N.B.:** If a random variable gets value $A$ with probability $p$ and value $B$ with probability $1 - p$, then the variance of that random variable is $(A - B)^2 \times p \times (1 - p)$.

### Question 8.1: The Formulation (15 points)

Complete the formulation of the problem of minimizing the uncertainty in the total profit of CYCOM. You may use the following values as given: \
$b_i$: profit if project $i$ is successful. \
$c_i$: start-up cost of project $i$. (A number greater than zero.) \
$a_i$: the constant term in the denominator of the probability of success of project $i$.

### Decision Variables:
$x_i$: The number of engineers assigned to project $i$ - non-negative continuous variables. \
$y$: Total number of engineers hired - non-negative integer variable.

### Objective Function:
$$ Minimize  \sum_{i = 1}^6 (b_i - c_i)^2  \times \frac{x_i}{(x_i + a_i)} \times (1 - \frac{x_i}{(x_i + a_i)}))$$

### Constraints:
$$ \sum_{i = 1}^6 (\frac{x_i \times b_i}{(x_i + a_i)} - \frac{c_i}{(x_i + a_i)} \times x_i ) \geq 1100$$
$$\sum_{i = 1}^6 x_i \leq y_i$$
$$ y \leq 25 $$
$$  x_i \geq 0 (for \: i = 1, \dots, 6)$$

### Question 8.2: The Code (15 points)

Implement the formulation you introduced in Question 8.1 in Python. Then, **find and report the minimum standard deviation of the total expected profit, the total expected profit, total number of engineers, and the number of engineers assigned to each project.**

In [3]:
SetupCost = [325, 200, 490, 125, 710, 240]
Profit = [1750, 700, 1300, 800, 1450, 1300]
Denom = [1.1, 0.5, 2.5, 1.6, 2.2, 2.4]
n = len(Profit)

# Create a new model.
model = gb.Model('Question8')
model.Params.LogToConsole = 0
model.params.NonConvex = 2

# Your code here:

# Variables
x = model.addVars(n, lb=0, name="Engineers assigned to each project")  
y = model.addVar(vtype=gb.GRB.INTEGER, lb=0, name="Total engineers")  
p = model.addVars(n, name="p") # Newly introduced variable p_i

# Objective Function (Min. Variance of total Profit)
variance = gb.quicksum((Profit[i] - SetupCost[i])**2 * p[i] * (1 - p[i]) for i in range(n))
model.setObjective(variance, gb.GRB.MINIMIZE)

# First Constraint: Expected Profit
expected_profit = gb.quicksum(p[i] * Profit[i] - (1 - p[i]) * SetupCost[i] for i in range(n)) - 150 * y
model.addConstr(expected_profit >= 1100)  # Expected profit at least $1.1 million

# Second Constraint: Sum of engineers assigned should not exceed the total number of engineers
model.addConstr(sum(x[i] for i in range(n)) <= y)

# Third Constraint: The total number of engineers should be less than or equal to 25
model.addConstr(y <= 25)

# Constraints for the new variables p_i
for i in range(n):
    model.addConstr(p[i] * (x[i] + Denom[i]) == x[i])

model.optimize()

if model.status == gb.GRB.OPTIMAL:
    minimum_std_deviation = variance.getValue()**0.5
    total_expected_profit = expected_profit.getValue()
    total_engineers = y.X
    engineers_per_project = [x[i].X for i in range(n)]
    print("Minimum Standard Deviation of Total Expected Profit (in $1000):", minimum_std_deviation)
    print("Total Expected Profit (in $1000):", total_expected_profit)
    print("Total Number of Engineers:", total_engineers)
    print("Number of Engineers Assigned to Each Project:", engineers_per_project)

Minimum Standard Deviation of Total Expected Profit (in $1000): 966.6988039104723
Total Expected Profit (in $1000): 1100.0000000000005
Total Number of Engineers: 14.0
Number of Engineers Assigned to Each Project: [4.0139180732148265, 1.3652279720005376, 3.0773523082981575, 1.5007858514946026, 3.641031119544171, 0.40168467544770425]
